#### LIVRABLE PROJET PYTHON

#### INPUT
1) Import des libraires

In [40]:
import os
from datetime import datetime 
import numpy as np
import pandas as pd
from IPython.display import display, clear_output
import plotly.graph_objects as go

2) Création des dataframes à partir des fichiers csv

In [41]:
def load_data_from_csv():

    notebook_dir = os.getcwd()        
                      #Chemin absolu du notebook
    data_dir = os.path.join(notebook_dir, "../data")    #Construction du chemin absolu vers le dossier 'data'
    csv_files = os.listdir(data_dir)                    #Liste des fichiers dans le dossier 'data'

    dataframes_dict = {}                                #Déclaration dictionnaire pour contenir les dataframes
    
    for file in csv_files:                              #Boucle pour chaque fichier .csv du dossier 'data' : 
        if file.endswith('.csv'):
            
            file_path = os.path.join(data_dir, file) 
            df = pd.read_csv(file_path)                         #Création du dataframe

            file_name = os.path.splitext(file)[0]
            dataframes_dict['df_' + str(file_name)] = df        #Renommage du dataframe avec préfixe "df_" + 'nom_du_fichier'
            
            df['timestamp'] = pd.to_datetime(df['timestamp'])   #Conversion 'timestamp' en type Datetime    

            print(f"df_{file_name}")                            #Affichage du df créé pour vérification
            print(df.head(3))   
            print("\n")

    return dataframes_dict

dataframes_dict = load_data_from_csv()

df_stmp
            timestamp    price  amount
0 2021-02-24 23:59:54  49754.0   0.753
1 2021-02-24 23:59:52  49754.0   0.116
2 2021-02-24 23:59:52  49754.0   0.104


df_lmax
                timestamp    price  amount
0 2021-02-24 23:59:59.691  49767.0    0.01
1 2021-02-24 23:59:42.786  49752.0    0.06
2 2021-02-24 23:59:42.785  49752.0    0.30


df_gmni
                timestamp     price    amount
0 2021-02-24 23:59:53.406  49773.07  0.003435
1 2021-02-24 23:59:53.406  49766.06  0.051690
2 2021-02-24 23:59:47.280  49746.16  0.122833


df_itbi
                timestamp     price  amount
0 2021-02-24 23:59:48.157  49753.50  0.0001
1 2021-02-24 23:59:45.463  49753.75  0.0004
2 2021-02-24 23:59:38.887  49734.50  0.0001


df_okcn
                timestamp     price  amount
0 2021-02-24 23:59:57.847  49724.93    0.02
1 2021-02-24 23:59:49.743  49730.33    0.02
2 2021-02-24 23:59:35.623  49706.87    0.02


df_bfnx
                timestamp    price  amount
0 2021-02-24 23:59:58.181  49716.0 

#### MANIPULATION DES DONNEES

1) Création d'un dataframe 'df_all' et intégration dans le dictionnaire

In [42]:

def global_df_creation(dataframes_dict): 
    
    dfs_to_concat = list(dataframes_dict.values())      #Identification des dataframes à concaténer
    print(f"Concaténation des dataframes :\n{list(dataframes_dict)}\n")
    
    df_all = pd.concat(dfs_to_concat, ignore_index=True)        #Concaténation verticale -> pas de perte de données avec la fonction concat, redéfinition des index pour avoir une clé unique par transaction
    print("'df_all' :")     
    display(df_all)     #Vérification : '2021-02-24 23:59:52.000' *2 dans le display = transactions conservées      

    all_dataframes_dict = dataframes_dict.copy()
    all_dataframes_dict['df_all'] = df_all
    print(f"Intégration dans le dictionnaire 'all_dataframes_dict' :\n{list(all_dataframes_dict)}\n")

    return all_dataframes_dict
    
all_dataframes_dict = global_df_creation(dataframes_dict) 


Concaténation des dataframes :
['df_stmp', 'df_lmax', 'df_gmni', 'df_itbi', 'df_okcn', 'df_bfnx', 'df_btrx', 'df_bfly', 'df_bnus', 'df_cbse', 'df_krkn']

'df_all' :


,timestamp,price,amount
0,2021-02-24 23:59:54.000,49754.0,0.753000
1,2021-02-24 23:59:52.000,49754.0,0.116000
2,2021-02-24 23:59:52.000,49754.0,0.104000
3,2021-02-24 23:59:49.000,49754.0,0.016000
4,2021-02-24 23:59:45.000,49754.0,0.011000
...,...,...,...
1136788,2021-02-24 00:00:11.182,48899.8,0.023270
1136789,2021-02-24 00:00:10.373,48899.9,0.200000
1136790,2021-02-24 00:00:07.818,48899.9,0.018278
1136791,2021-02-24 00:00:02.351,48899.9,0.002045


Intégration dans le dictionnaire 'all_dataframes_dict' :
['df_stmp', 'df_lmax', 'df_gmni', 'df_itbi', 'df_okcn', 'df_bfnx', 'df_btrx', 'df_bfly', 'df_bnus', 'df_cbse', 'df_krkn', 'df_all']



#### CALCUL

1) Définition des paramètres de calcul

In [43]:
def setup_values():

    frequency = '60min'
    vwmp_type = 'lower' 
    
    return frequency, vwmp_type 

2) Agrégation des données et calcul selon paramètres sélectionnés

In [44]:
def process_data(all_dataframes_dict, frequency, vwmp_type):                     #PROCESS DE CALCUL :                                                    
    
    print("...calcul en cours")
    
    aggregated_dict = {}        #Données initiales aggrégées (1)
    calculated_dict = {}        #Données calculées (vwap, ecart_type, vwmp) (2)
    cleaned_dict = {}           #Données nettoyées (suppression des lignes vides) (3)
    compilated_dict = {}        #Compilation des résultats (création du 'df_synthese') (4)                  

    for key, df in all_dataframes_dict.items():                                         

        exchange_name = (str(key)).split("df_")[1]                                  
        
        aggregated_df = aggregate_data(df, frequency)     #(1)Aggréggation des données initiales
        aggregated_dict[key] = aggregated_df 
        
        calculated_df = calculate_metrics(df, frequency, vwmp_type, aggregated_df, exchange_name)       #(2)Calcul des métriques
        calculated_dict[key] = calculated_df 
                
        cleaned_df = clean_data(calculated_df)            #(3)Suppression des lignes vides
        cleaned_dict[key] = cleaned_df

    compilated_dict = compilate_data(cleaned_dict)        #(4)Compilation des résultats dans un 'df_synthese'
      
    return aggregated_dict, calculated_dict, cleaned_dict, compilated_dict

def aggregate_data(df, frequency):                                               #(1) Aggrégation des données initiales (Price, Amount)                  
    
    df['weighted_volume'] = (df['price'] * df['amount'])        #Calcul du Weighted_Volume de chaque transaction avant agréggation 

    aggregated_df = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).agg({    #Fonctions d'agrégation pour chaque colonnes
        'price': ['sum', 'first', 'max', 'min', 'last'],   
        'amount': 'sum',
        'weighted_volume': 'sum'
    })

    aggregated_df.columns = ['price', 'price_open', 'price_high', 'price_low', 'price_close','amount','weighted_volume']        #Renommage des colonnes
    
    return aggregated_df

def calculate_metrics(df, frequency, vwmp_type, aggregated_df, exchange_name):   #(2) Calcul des métriques (VWAP, VWMP, ecart_type)                      
    
    calculated_df = aggregated_df     

    calculated_df[f'{exchange_name}_vwap'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).apply(calculate_vwap)                                           #Calcul du Volume Weighted Average Price [VWAP] 
    calculated_df['ecart_type'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).apply(calculate_ecart_type)                                                #Calcul de l'écart_type 
    calculated_df[f'vwmp_{vwmp_type}'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).apply(lambda group: pd.Series(calculate_vwmp(group, vwmp_type)))    #Calcul du Volume Weighted Median Price [VWMP (lower or upper)] 

    return calculated_df 

def calculate_vwap(group):                                                       #(2.1) Calcul du Volume Weighted Average Price [VWAP]                   
    sum_price_amount = group['weighted_volume'].sum()                                           
    sum_amount = group['amount'].sum()

    if sum_amount != 0:
        return sum_price_amount / sum_amount 
    else : 
        return 0    

def calculate_ecart_type(group):                                                 #(2.2) Calcul de l'écart_type                                           
    
    ecart_type = np.nanstd(group['price'])      #'np.nanstd' pour exclure les valeurs nulles dans le calcul de l'écart type 
    if ecart_type != np.nan :
        return ecart_type
    else : 
        return 0        #écart type à zéro dans le cas d'un échantillon de taille 1 (une seule valeur de 'price' sur la période)

def calculate_vwmp(group, vwmp_type):                                            #(2.3) Calcul du Volume Weighted Median Price [VWMP (lower or upper)]   

    series_sorted = group.sort_values('amount')                         #Tri du volume par ordre croissant
    series_sorted['cumul_amount'] = series_sorted['amount'].cumsum()    #Calcul du volume cumulé
    total_volume_median = series_sorted['cumul_amount'].max() / 2       #Calcul de la médiane
    
    #--------
    if vwmp_type == 'lower':    #LOWER
       
        lower_cumulative_volume = series_sorted[series_sorted['cumul_amount'] <= total_volume_median]   #Filtrage de la série pour conserver la partie inférieure (>=) du volume cumulé 
       
        if not lower_cumulative_volume.empty:   #Si la taille de l'échantillon est >> 2
           
            max_cumul_amount_index = lower_cumulative_volume['cumul_amount'].idxmax()   #Identification de la ligne correspondante au volume cumulé maximum de la série filtrée (ensemble inférieure)
            vwmp_lower = group.loc[max_cumul_amount_index, 'price']                     #Déduction du prix médian bas pondéré par le volume
            return vwmp_lower  
        
        else : 
            return group['price'].mean()    #Cas particuliers (mesuré x3 occurences) soit (vwmp = price si échantillon = 1 ;  vwmp = 0 si échantillon = 0) -> utilisation de la fonction mean pour renvoyer ce résultat

    #-- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - #symétrie

    if vwmp_type == 'upper':    #UPPER
    
        upper_cumulative_volume = series_sorted[series_sorted['cumul_amount'] <= total_volume_median]   #Filtrage de la série pour conserver la partie inférieure (>=) du volume cumulé 
    
        if not upper_cumulative_volume.empty:   #Si la taille de l'échantillon est >> 2
       
            min_cumul_amount_index = upper_cumulative_volume['cumul_amount'].idxmin()   #Identification de la ligne correspondante au volume cumulé maximum de la série filtrée (ensemble inférieure)
            vwmp_upper = group.loc[min_cumul_amount_index, 'price']                     #Déduction du prix médian bas pondéré par le volume
            return vwmp_upper  
    
        else : 
            return group['price'].mean()    #Cas particuliers (mesuré x3 occurences) soit (vwmp = price si échantillon = 1 ;  vwmp = 0 si échantillon = 0) -> utilisation de la fonction mean pour renvoyer ce résultat
    
    #----------
    else:
        raise ValueError("Mode non valide. Veuillez spécifier 'lower' ou 'upper'.")

def clean_data(calculated_df):                                                   #(3) Supprime les lignes vides (intervalle de temps sans transaction)   
    
    cleaned_df = calculated_df.copy()                                           #Copie pour ne pas modifier le df original
    cleaned_df = cleaned_df[~cleaned_df.apply(                                  #Filtrage des lignes où toutes les valeurs sont nulles
        lambda row: all(val == 0.0 or pd.isnull(val) for val in row), axis=1    
    )]               
    return cleaned_df

def compilate_data(cleaned_dict):                                                #(4) Assemblage des résultats dans un dataframe 'df_synthese'           
    
    df_synthese = None    #Déclaration d'un dataframe vide
    
    for key, df in cleaned_dict.items():        
        
        exchange_name = (str(key)).split("df_")[1]           #Extraction du nom de l'exchange dans le nom du dataframe
                                
        if f'{exchange_name}_vwap' in df.columns:            #Check si la colonne 'vwap' existe dans le dataframe 
            
            vwap_column = df[f'{exchange_name}_vwap']        #Identifie la colonne 'wvap' de l'exchange
                        
            if df_synthese is None:                          #1ère itération
               df_synthese = vwap_column.to_frame()          #Copie de la colonne 'vwap' de l'exchange et intégration au dataframe de synthese
            else:
               df_synthese = pd.concat([df_synthese, vwap_column], axis=1)  #itération >= 2 : concaténe la colonne 'wvap' de l'exchange au dataframe de synthese
    
    compilated_dict = cleaned_dict.copy()                #Duplication du dictionnaire d'entrée
    compilated_dict['df_synthese'] = df_synthese         #Intégration du dataframe 'synthese' dans le nouveau dictionnaire
        
    return compilated_dict

3) Visualisation des données

In [45]:
def visualisation(cleaned_dict):
    
    candlestick_dict = {}    

    for key, df in reversed(list(cleaned_dict.items())):  #Lecture en reverse pour afficher le df_all en 1er
    
        exchange_name = (str(key)).split("df_")[1]        #Extraction du nom de l'exchange
        vwmp_column_index = 9                             #Index de la colonne 'vwmp_XXXer'(10ème colonne)
        
        candlestick = go.Figure(data=[go.Candlestick(x=df.index,                    #Création d'un graphique en chandelier japonais 
                                                      open=df['price_open'],
                                                      high=df['price_high'],
                                                      low=df['price_low'],
                                                      close=df['price_close']
                                                     )])

        candlestick.update_layout(title=f'Exchange - [{exchange_name}]', yaxis_title='Price', height=300, width=800, margin=dict(l=70, r=50, t=50, b=20))      #Mise en forme du graphique
        candlestick.add_trace(go.Scatter(x=df.index, y=df[f'{exchange_name}_vwap'], mode='lines', name='VWAP', line_color='blue'))                             #Ajout courbe du Volume Weighted Average Price [VWAP]
        candlestick.add_trace(go.Scatter(x=df.index, y=df.iloc[:, vwmp_column_index], mode='lines', name=df.columns[vwmp_column_index], line_color='grey'))    #Ajout courbe du Volume Weighted Median Price [VWMP] avec nom de série associé
        
        candlestick_dict[exchange_name] = candlestick

    for exchange, candlestick in candlestick_dict.items(): 
        
        display(candlestick)

    return candlestick_dict         

4) Concaténation des données et export csv

In [46]:

def export_csv(compilated_dict, frequency, vwmp_type):      #Fonction en cas de clic sur le wigdget 'Exporter en csv'

    notebook_dir = os.getcwd()                                                                                              #Chemin du notebook
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")                                                                      #Timestamp du lancement export
    export_folder = (notebook_dir) + '/../output/Export_' + (now) + '_freq-'+ (frequency) + '_vwmp-' + (vwmp_type) +'/'     #Création d'un dossier 'export' horodaté avec les paramètres sélectionnées à l'export
    os.mkdir(export_folder)                                                                                                 #Création du dossier 'Export_YYYY-MM-DD_HH-MM-SS_freq-Xmin_vwmp-X'
                                                                                                    
    export_sucess = False 

    try:
        for key, df in compilated_dict.items():   

            filename = (f"{key}.csv")                   #ID du fichier csv = nom du dataframe
            file_path = export_folder + filename        #Chemin de l'enregistrement
            df.to_csv(file_path, index=True)            #enregistre le dataframe en fichier .csv

        export_success = True
        print(f"Export des fichiers csv avec succès (voir arborescence ci-dessous): {export_folder}")
        return export_sucess
    
    except:
        export_success = False
        print("Erreur lors de l'export")
        return export_sucess 


#### RESULTATS

Dans le cas où les widgets ne fonctionnerait pas dans le jupyter notebook :

1) Lancer l'aggrégation avec les paramètres souhaités et visualiser les résultats :

In [49]:
#1) Définir les paramètres
frequency, vwmp_type = setup_values()

#2) Calculer 
aggregated_dict, calculated_dict, cleaned_dict, compilated_dict = process_data(all_dataframes_dict, frequency, vwmp_type)  
clear_output()

#3) Visualiser
#candlestick_dict = visualisation(cleaned_dict)

#4) Exporter
#export_success = export_csv(compilated_dict, frequency, vwmp_type)

2) Exporter les données avec les paramètres sélectionnés

Arborescence du projet : 

In [48]:
##  .
##  ├── data
##  │   ├── bfly.csv
##  │   ├── bfnx.csv
##  │   ├── bnus.csv
##  │   ├── btrx.csv
##  │   ├── cbse.csv
##  │   ├── gmni.csv
##  │   ├── itbi.csv
##  │   ├── krkn.csv
##  │   ├── lmax.csv
##  │   ├── okcn.csv
##  │   └── stmp.csv
##  ├── livrable
##  │   └── livrable.ipynb
##  ├── output
##  │   ├── Export_YYYY-MM-DD_HH-MM-SS_freq-Xmin_vwmp-X
##  │   │   ├── df_all.csv
##  │   │   ├── df_bfly.csv
##  │   │   ├── df_bfnx.csv
##  │   │   ├── df_bnus.csv
##  │   │   ├── df_btrx.csv
##  │   │   ├── df_cbse.csv
##  │   │   ├── df_gmni.csv
##  │   │   ├── df_itbi.csv
##  │   │   ├── df_krkn.csv
##  │   │   ├── df_lmax.csv
##  │   │   ├── df_okcn.csv
##  │   │   ├── df_stmp.csv
##  │   │   └── synthese.csv
##  └── readme.txt